# Machine Translation della Lezione4, in Lezione5 Huggingface tokenizer invece che Spacy
vedi Notebook prof, ma attento che su questo hai preso appunti

In [1]:
!pip install tokenizers

In [62]:
import random
from tqdm import tqdm
from collections import Counter
from tensorflow.data import Dataset
import tensorflow as tf
import keras
from keras.layers import GRUCell, Embedding, Attention, Dense
import json
from tokenizers import Tokenizer
from tokenizers.normalizers import  Lowercase
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer

In [3]:
!wget https://www.manythings.org/anki/ita-eng.zip

--2023-09-20 14:20:06--  https://www.manythings.org/anki/ita-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110, 198.41.0.4, 199.9.14.201, ...
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8220355 (7.8M) [application/zip]
Saving to: ‘ita-eng.zip.1’

ita-eng.zip.1       100%[===================>]   7.84M  2.92MB/s    in 2.7s    

2023-09-20 14:20:10 (2.92 MB/s) - ‘ita-eng.zip.1’ saved [8220355/8220355]



In [4]:
!unzip ita-eng.zip

Archive:  ita-eng.zip
replace ita.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [63]:
UNK = "<UNK>"
BOS = "<BOS>"
EOS = "<EOS>"
PAD = "<PAD>"

In [64]:
PAD_IDX = 0
UNK_IDX = 1
BOS_IDX = 2
EOS_IDX = 3

# tokenizzazione

In [65]:
special_tokens = [PAD, UNK, BOS, EOS]

In [66]:
en_tokenizer = Tokenizer(WordPiece(unk_token=UNK))
en_tokenizer.normalizer = Lowercase()
en_tokenizer.pre_tokenizer = Whitespace()

it_tokenizer = Tokenizer(WordPiece(unk_token=UNK))
it_tokenizer.normalizer = Lowercase()
it_tokenizer.pre_tokenizer = Whitespace()

In [67]:
def lang_iterator(lang="en"):
    with open("ita.txt", "r") as f:
        while line:=f.readline()[:-1]:
            en_sent, it_sent, _ = line.split("\t")
            yield en_sent if lang=="en" else it_sent

In [68]:
en_tokenizer.train_from_iterator(lang_iterator("en"), trainer=WordPieceTrainer(vocab_size=10_000, special_tokens=special_tokens))
it_tokenizer.train_from_iterator(lang_iterator("it"), trainer=WordPieceTrainer(vocab_size=10_000, special_tokens=special_tokens))

In [69]:
en_vocab = en_tokenizer.get_vocab()
it_vocab = it_tokenizer.get_vocab()
en_vocab_inv = {v:k for k, v in en_vocab.items()}
it_vocab_inv = {v:k for k, v in it_vocab.items()}

# split train-validation-test

In [20]:
def split_file(filepath):
    with open(filepath, "r") as f:
        lines = f.readlines()

    num_rows = len(lines)
    shuffled_row_idxs = random.sample(list(range(num_rows)), k=num_rows)

    train_idxs = shuffled_row_idxs[:int(num_rows*0.6)]
    val_idxs = shuffled_row_idxs[int(num_rows*0.6):int(num_rows*0.8)]
    test_idxs = shuffled_row_idxs[int(num_rows*0.8):]

    # train
    with open("train.txt", "w") as f:
        f.writelines([lines[idx] for idx in train_idxs])

    # validation
    with open("val.txt", "w") as f:
        f.writelines([lines[idx] for idx in val_idxs])

    # test
    with open("test.txt", "w") as f:
        f.writelines([lines[idx] for idx in test_idxs])

In [13]:
split_file("ita.txt")

# Generatori

In [48]:
def dataset_generator(filepath, en_dict, it_dict, en_tokenizer, it_tokenizer):
    def gen():
        with open(filepath, "r") as f:
            while (line := f.readline()) is not None:
                en_sentence, it_sentence, *_ = line.split("\t")
                en_sentence_tokenized = en_tokenizer.encode(en_sentence).tokens
                it_sentence_tokenized = it_tokenizer.encode(it_sentence).tokens
                src_sentence_tokenized = en_sentence_tokenized + [EOS]
                tgt_sentence_in_tokenized = [BOS] + it_sentence_tokenized
                tgt_sentence_out_tokenized = it_sentence_tokenized.copy() + [EOS]

                src_sentence_encoded = [en_dict.get(token, en_dict[UNK]) for token in src_sentence_tokenized]
                tgt_sentence_in_encoded = [it_dict.get(token, it_dict[UNK]) for token in tgt_sentence_in_tokenized]
                tgt_sentence_out_encoded = [it_dict.get(token, it_dict[UNK]) for token in tgt_sentence_out_tokenized]

                yield (src_sentence_encoded, tgt_sentence_in_encoded), tgt_sentence_out_encoded # input, target in e target_out

    return gen

In [15]:
# for (src, tgt_in), tgt_out in dataset_generator("val.txt", en_vocab, it_vocab, Tokenizer("en"), Tokenizer("it")):
#     print(src)
#     print(tgt_in)
#     print(tgt_out)
#     break

In [49]:
trainset = Dataset.from_generator(
    generator=dataset_generator("train.txt", en_vocab, it_vocab, en_tokenizer, it_tokenizer),
    output_signature=(
        (tf.TensorSpec(shape=(None,), dtype=tf.int32), tf.TensorSpec(shape=(None,), dtype=tf.int32)),
        tf.TensorSpec(shape=(None,), dtype=tf.int32))
)

valset = Dataset.from_generator(
    generator=dataset_generator("val.txt", en_vocab, it_vocab, en_tokenizer, it_tokenizer),
    output_signature=(
        (tf.TensorSpec(shape=(None,), dtype=tf.int32), tf.TensorSpec(shape=(None,), dtype=tf.int32)),
        tf.TensorSpec(shape=(None,), dtype=tf.int32))
)

testset = Dataset.from_generator(
    generator=dataset_generator("test.txt", en_vocab, it_vocab, en_tokenizer, it_tokenizer),
    output_signature=(
        (tf.TensorSpec(shape=(None,), dtype=tf.int32), tf.TensorSpec(shape=(None,), dtype=tf.int32)),
        tf.TensorSpec(shape=(None,), dtype=tf.int32))
)

In [50]:
trainset = trainset.shuffle(buffer_size=1000, reshuffle_each_iteration=True)
trainset = trainset.padded_batch(batch_size=32)

In [51]:
class TextEncoderDecoder:
    def __init__(self, en_vocab, en_vocab_inv, it_vocab, it_vocab_inv):
        self.en_vocab = en_vocab
        self.en_vocab_inv = en_vocab_inv
        self.it_vocab = it_vocab
        self.it_vocab_inv = it_vocab_inv
        self.nlp_it = spacy.load("it_core_news_md")
        self.nlp_en = spacy.load("en_core_web_md")

    def encode(self, text, language):
        vocab = self.en_vocab if language == "en" else self.it_vocab
        nlp = self.nlp_en if language == "en" else self.nlp_it
        tokenized_text = [token.text.lower() for token in nlp(text)]
        return [vocab.get(token, vocab[UNK]) for token in tokenized_text]

    def decode(self, coded_text, language):
        vocab_inv = self.en_vocab_inv if language == "en" else self.it_vocab_inv
        return [vocab_inv[code] for code in coded_text]

# Modello

In [52]:
class Encoder(keras.Model):
    def __init__(self, vocabulary_size, embedding_size, recurrent_layers, recurrent_units, **kwargs):
        """
        args
        ----
        - vocabulary_size (int): including special tokens (<BOS>, <EOS>, <UNK>)
        - embedding_size (int): dimensione dello spazio degli embedding

        """
        super().__init__(**kwargs)

        # embedding
        # 0 index mean padding
        # input: batch_size x seq_length
        self.embedding = Embedding(
            vocabulary_size, embedding_size, mask_zero=True) # costruisce matrice
        # output: batch_size x seq_length x embedding_size (ogni frase rappr da una matrice)

        gru_cells = [GRUCell(recurrent_units) for _ in range(recurrent_layers)]

        stacked_cells = tf.keras.layers.StackedRNNCells(gru_cells)
        self.gru_layer = tf.keras.layers.RNN(stacked_cells, return_state=True, return_sequences=True)

    def call(self, data, training=None):
        # data.shape: batch_size x seq_len
        x = self.embedding(data, training=training)
        # data.shape: batch_size x seq_len x embedding_size
        output, *state = self.gru_layer(x, training=training)

        return output, state

In [53]:
# Esempio
encoder = Encoder(vocabulary_size=10_000,
       embedding_size = 128,
       recurrent_layers = 2,
       recurrent_units = 128
       )

sentences = tf.random.uniform([2,10], 0, 100, tf.int32)
output, state = encoder(sentences)
output.shape, len(state)

(TensorShape([2, 10, 128]), 2)

In [28]:
state[1].shape # batch_size x 128 (embedding_size)

TensorShape([2, 128])

In [54]:
class Decoder(keras.Model):
    def __init__(self, vocabulary_size, embedding_size, recurrent_layers, recurrent_units, **kwargs):
        super().__init__(**kwargs)

        self.recurrent_layers = recurrent_layers
        self.recurrent_units = recurrent_units

        self.embedding = Embedding(
            vocabulary_size, embedding_size, mask_zero=True)

        gru_cells = [GRUCell(recurrent_units) for _ in range(recurrent_layers)]

        stacked_cells = tf.keras.layers.StackedRNNCells(gru_cells)
        self.gru_layer = tf.keras.layers.RNN(stacked_cells, return_state=True)


    def call(self, target_in, encoder_output, rnn_state, training=None):
        # target_in.shape = batch_size
        # target_in_embedded.shape = batch_size x embedding_dim
        target_in_embedded = tf.squeeze(self.embedding(tf.expand_dims(target_in, axis=1), training=training))

        # creo il contesto
        # context.shape = batch_size x recurrent_units

        context = encoder_output[:, -1, :] # il contesto è l'output dell'encoder


        # concateno embedding della parola e contesto
        # word_and_context.shape = batch_size x 1 x (recurrent_units+embedding_dim)
        word_and_context = tf.expand_dims(tf.concat([target_in_embedded, context], axis=1), axis=1)

        # output.shape = batch_size x recurrent_units
        output, *state = self.gru_layer(word_and_context, initial_state=rnn_state, training=training)

        return output, state

In [70]:
MAX_SENTENCE_LENGTH = 50

In [82]:
class EncoderDecoder(keras.Model):
    def __init__(self, vocabulary_size, embedding_size, recurrent_layers, recurrent_units, **kwargs):
        super().__init__(**kwargs)

        assert embedding_size == recurrent_units, "must be embedding_size == recurrent_units"

        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size
        self.recurrent_layers = recurrent_layers
        self.recurrent_units = recurrent_units


        self.encoder = Encoder(vocabulary_size, embedding_size,
                               recurrent_layers, recurrent_units)
        self.decoder = Decoder(vocabulary_size, embedding_size, recurrent_layers,
                               recurrent_units)

        self.dense = Dense(vocabulary_size, "softmax")

    def call(self, data, training=None, max_sentence_length=MAX_SENTENCE_LENGTH):
        # unpack data
        src_sentences, dst_sentences = data

        probs = []

        # encoder call
        # encoder_output.shape = batch x len_sentences x encoder_recurrent_units
        encoder_output, encoder_state = self.encoder(src_sentences, training=training)

        seq_len = max_sentence_length if not training else tf.shape(dst_sentences)[1]
        decoder_state = encoder_state
        for i in range(seq_len):
            if not training:
                if i == 0:
                    decoder_input = tf.fill(dims=[tf.shape(src_sentences)[0]], value=BOS_IDX)
                else:
                    decoder_input = tf.math.argmax(probs[-1], axis=1)
            else:
                decoder_input = dst_sentences[:, i]
            decoder_output, decoder_state = self.decoder(decoder_input, encoder_output, decoder_state, training=training)
            probs.append(self.dense(decoder_output))

        # decoder_output.shape = batch_size x max_sentence_length x vocabulary_size
        return tf.stack(probs, axis=1)

In [83]:
def custom_loss(y_true, y_pred):
    y_true_reshaped = tf.reshape(y_true, [-1])
    y_pred_reshaped = tf.reshape(y_pred, [-1, y_pred.shape[-1]])
    scc = tf.keras.losses.SparseCategoricalCrossentropy(
        reduction=tf.keras.losses.Reduction.NONE)
    results = scc(y_true_reshaped, y_pred_reshaped)
    mask = tf.cast(y_true_reshaped != 0, tf.float32)
    return tf.reduce_sum(results*mask) / tf.reduce_sum(mask)

# Addestramento

In [61]:
next(iter(trainset)) # input, target in, target output

((<tf.Tensor: shape=(32, 18), dtype=int32, numpy=
  array([[1805,  154,   29, 2085,   14,    3,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0],
         [ 128,  535,  205,  507,  716,  137,  128,  486,   13, 5966, 1551,
          2224,  156,  171,  283,  529,    4,    3],
         [ 154,    9,  231,  472,  752,   14,    3,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0],
         [ 521,   37, 1389,  187, 1871,   28,    3,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0],
         [ 194,  140,  156,  289,  418,   28,    3,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0],
         [  37,    9,   41,  938,  283,  446,  642,   14,    3,    0,    0,
             0,    0,    0,    0,    0,    0,    0],
         [  37,  918,  411,  986,  324,  472,   14,    3,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0],
         [ 128,  952,  137,  152,  128, 5610,  

In [84]:
encoder_decoder = EncoderDecoder(vocabulary_size=10_000, embedding_size=256, recurrent_layers=2, recurrent_units=256)

In [85]:
encoder_decoder.compile(optimizer=keras.optimizers.Adam(), loss=custom_loss, run_eagerly=True)

In [86]:
encoder_decoder.fit(x=trainset, steps_per_epoch=100, epochs=100, initial_epoch=0)

Epoch 1/100
 12/100 [==>...........................] - ETA: 1:55 - loss: 8.6067

KeyboardInterrupt: 

In [ ]:
encoder_decoder.save_weights("weights_100.h5")

# Text generation

In [ ]:
sentence = "i am a good guy and my home is beautiful"
nlp = spacy.load("en_core_web_md")
tokens = [t.text.lower() for t in nlp(sentence)]
idxs = [en_vocab.get(t, en_vocab[UNK]) for t in tokens]
translation = encoder_decoder.generate(tf.reshape(tf.constant(idxs), [1, -1]))
print(" ".join([it_vocab_inv[idx] for idx in translation]))

sono un insegnante di francese , non sono bravo in un amico . <EOS> . <EOS> . <EOS> . <EOS>


# Altro
Spiegazione Matrice di Embedding

In [2]:
from keras.layers import Embedding
import tensorflow as tf

2023-09-20 14:46:25.555001: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-20 14:46:25.619760: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-20 14:46:25.620416: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-20 14:46:26.655965: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
embedding = Embedding(input_dim=10_000, output_dim=512, mask_zero=True)
# vocab_size, dimensione del vettore parola

In [4]:
sentences = tf.random.uniform([2, 10], 0, 100, tf.int32) # 2 frasi lunghe 10 parole (interi da 0 a 100)
embedded_sentences = embedding(sentences)

In [5]:
embedding.get_weights()[0].shape

(10000, 512)

In [6]:
embedded_sentences.shape # ogni frase, 10 token, ciascun token in un vettore da 512 float

TensorShape([2, 10, 512])

In [7]:
embedded_sentences[0][0].shape # frase 1, token 1 (è un vettore lungo 512)

TensorShape([512])

In [8]:
embedded_sentences[0][0]

<tf.Tensor: shape=(512,), dtype=float32, numpy=
array([-2.50312928e-02,  1.83144473e-02, -6.85974211e-03, -2.88465749e-02,
        4.67611887e-02, -2.56500840e-02, -2.82975193e-02, -4.49316502e-02,
       -1.24897584e-02,  8.83637741e-03,  4.66066636e-02, -3.94867174e-02,
       -2.75575761e-02,  1.59662701e-02,  3.64848636e-02,  1.82839967e-02,
       -4.03087512e-02, -2.99840216e-02,  2.91905552e-03, -9.92680714e-03,
       -9.50132683e-03, -1.70045868e-02,  3.31894271e-02, -4.06564251e-02,
        2.63581388e-02,  8.62586498e-03, -3.44862714e-02, -4.37368043e-02,
       -4.69430201e-02,  4.11962382e-02, -1.64128169e-02,  3.92797701e-02,
        8.78310204e-03,  3.46171297e-02,  2.92851590e-02, -3.85824814e-02,
        1.38792656e-02, -2.36201771e-02,  2.26876773e-02,  4.99265231e-02,
       -1.63786300e-02, -4.24662344e-02,  3.01709026e-03, -8.41565058e-03,
       -2.91646607e-02, -4.14339192e-02, -7.65029341e-03,  2.03201063e-02,
        3.95124666e-02, -2.64043733e-03,  1.53302066

In [9]:
embedding.get_weights()[0][0]

array([-3.17471176e-02,  1.21394880e-02,  2.51705386e-02,  3.73667739e-02,
       -1.82010904e-02, -3.31496149e-02, -4.23372909e-03,  2.48690955e-02,
        3.16639207e-02, -4.75274920e-02,  3.64082493e-02,  1.34738125e-02,
       -2.69374494e-02,  1.45352371e-02,  3.54732983e-02,  1.10796466e-02,
       -2.89683100e-02, -1.54553652e-02,  1.92291774e-02, -2.38754749e-02,
        2.51053087e-02,  5.17425686e-03, -1.40281767e-03,  1.62094496e-02,
        4.55679931e-02,  3.00576948e-02, -8.76331329e-03, -3.21155936e-02,
        1.78758390e-02, -2.60051247e-02,  1.10275745e-02, -2.50389706e-02,
       -1.36180967e-03, -2.39867568e-02, -2.47322563e-02,  2.42419951e-02,
        2.50582956e-02, -3.39053422e-02, -4.07314412e-02, -4.81655262e-02,
        1.61961205e-02,  4.51124050e-02, -4.22927737e-02, -2.66074017e-03,
       -3.63420695e-04, -3.25978026e-02,  2.40233801e-02,  1.13788620e-02,
        3.56761850e-02,  3.81277092e-02, -2.86096819e-02, -2.03917865e-02,
        1.59296282e-02,  

In [10]:
# attributo mask_zero=True, appiccica una mask
embedded_sentences._keras_mask # i token di padding vengono ignorati?

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>

In [ ]:
Nota
Paperspace: https://www.paperspace.com/